# A Comparison of GaussianProcesses vs. GaussianProcesses2

In [1]:
using GaussianProcesses, GaussianProcesses2
using BenchmarkTools
using Plots; pyplot()

Plots.PyPlotBackend()

## GaussianProcesses

In [101]:
# Training data
srand(1989874)
ps1 = []
sigs1 = []
l = 2.0;                            # length scale
ν = 0.05                            # noise variance
m1 = MeanZero()                         #Zero mean function
k1 = SE(log(l),log(1.0))                #Squared exponential kernel (note that hyperparameters are on the log scale)
for n in [1,2,3,4,5,10,15,20,30,50,100,200]
    X = 2.5 * ones(n);                   # design points
    y = sin.(X) + sqrt(ν)*randn(n);     # observed objective values
    X_pred = collect(linspace(0.0,2π,20));
    gp1 = GP(X,y,m1,k1,log(sqrt(ν)))
    μ1, S = predict_f(gp1, X_pred)
    sig1 = sqrt.(S)
    push!(sigs1, sig1[1])
    p = plot(X_pred, μ1, ribbon=sig1, ylim=(0.0,1.2))
    plot!(p, X, y, seriestype=:scatter, ylim=(0.0,1.2))
    push!(ps1, p)
end
animate(ps1, "./gp1_repeated.gif"; fps=1)

INFO: Saved animation to /Users/rlee18/.julia/v0.6/CBandits/notebooks/gp1_repeated.gif


Plots.AnimatedGif("/Users/rlee18/.julia/v0.6/CBandits/notebooks/gp1_repeated.gif")

In [102]:
sigs1

12-element Array{Any,1}:
 0.894634
 0.891909
 0.890968
 0.890492
 0.890204
 0.889624
 0.88943 
 0.889332
 0.889234
 0.889156
 0.889097
 0.889067

## GaussianProcesses2

In [103]:
srand(1989874)
ps2 = []
sigs2 = []
l = 2.0;                            # length scale
ν = 0.05                            # noise variance
m2 = ZeroMean()  # constant zero mean function
k2 = SquaredExponential(l);  # squared exponential kernel
for n in [1,2,3,4,5,10,15,20,30,50,100,200]
    X = 2.5 * ones(n);                   # design points
    y = sin.(X) + sqrt(ν)*randn(n);     # observed objective values
    X_pred = collect(linspace(0.0,2π,20));
    gp2 = GaussianProcess(m2,k2,X,y,ν)
    μ2, v2 = GaussianProcesses2.predict(gp2, X_pred)
    sig2 = sqrt.(v2)
    push!(sigs2, sig2[1])
    p = plot(X_pred, μ2, ribbon=sig2)
    plot!(p, X, y, seriestype=:scatter)
    push!(ps2, p)
end
animate(ps2, "./gp2_repeated.gif"; fps=1)

INFO: Saved animation to /Users/rlee18/.julia/v0.6/CBandits/notebooks/gp2_repeated.gif


Plots.AnimatedGif("/Users/rlee18/.julia/v0.6/CBandits/notebooks/gp2_repeated.gif")

In [104]:
sigs2

12-element Array{Any,1}:
 0.894634
 0.891909
 0.890968
 0.890492
 0.890204
 0.889624
 0.88943 
 0.889332
 0.889234
 0.889156
 0.889097
 0.889067

### Manual testing

In [105]:
n = 3
X = 2.5 * ones(n);                   # design points
y = sin.(X) + sqrt(ν)*randn(n);     # observed objective values
X_pred = [2.5]
gp2 = GaussianProcess(m2,k2,X,y,ν)

GaussianProcesses2.GaussianProcess(GaussianProcesses2.ConstMean(0.0), GaussianProcesses2.SquaredExponential(4.0), [2.5, 2.5, 2.5], [0.927593, 0.43596, 0.455532], 0.05)

In [106]:
using GaussianProcesses2: μ, K
m, k, ν = gp2.m, gp2.k, gp2.ν
tmp = K(X_pred, gp2.X, k) / (K(gp2.X, gp2.X, k) + ν*I)

1×3 Array{Float64,2}:
 0.327869  0.327869  0.327869

In [107]:
S = K(X_pred, X_pred, k) - tmp*K(gp2.X, X_pred, k)

1×1 Array{Float64,2}:
 0.0163934

In [108]:
K(gp2.X, X_pred, k)

3×1 Array{Float64,2}:
 1.0
 1.0
 1.0

In [109]:
K(gp2.X, gp2.X, k)

3×3 Array{Float64,2}:
 1.0  1.0  1.0
 1.0  1.0  1.0
 1.0  1.0  1.0

In [110]:
A = (K(gp2.X, gp2.X, k) + ν*I)

3×3 Array{Float64,2}:
 1.05  1.0   1.0 
 1.0   1.05  1.0 
 1.0   1.0   1.05

In [111]:
rank(A)

3